# Model training

In [1]:
## Mathematical and statistical operations
import pandas as pd
from pandas import DataFrame
import numpy as np
from collections import defaultdict, Counter

## Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from geopy.distance import geodesic

## Metrics
from sklearn.metrics import classification_report, f1_score, roc_auc_score

## Visualization
import folium
from IPython.display import display, clear_output
from datetime import timedelta
import io
from PIL import Image
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import os
from time import sleep as time_sleep

## Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
def make_stid_unique(df: DataFrame, num: int) -> DataFrame:
    ## change each stid to be the appended version with _num
    df['stid'] = df['stid'].astype(str) + '_' + str(num)
    return df

def print_model_eval(model, X_test, y_test):
    print("Combined Model Classification Report:")
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)
    print(classification_report(y_test, y_pred))

    ## AOC_roc score
    roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')
    print(f"ROC AUC Score: {roc_auc:.4f}")
    
    
# new_headers = {
#         'stid',
#         'duration_seconds',
#         'total_distance_miles',
#         'mean_speed_mph',
#         'area_span_miles',
#         'lat_variation',
#         'lon_variation',
#         'behavior'
#     }

def extract_features(df: DataFrame) -> DataFrame:
    df = df.sort_values('time')
    coords = df[['latitude', 'longitude']].values
    df['time'] = pd.to_datetime(df['time'], errors='coerce')
    times = pd.to_datetime(df['time'])
    
    ## Total distance travelled
    distances = [geodesic(coords[i], coords[i+1]).miles for i in range(len(coords)-1)]
    total_distance = sum(distances)
    
    ## Time duration
    duration = (times.iloc[-1] - times.iloc[0]).total_seconds()    
    
    ## Mean speed (miles per hour)
    mean_speed = (total_distance / duration) * 3600 if duration > 0 else 0
    
    ## Bounding box, area covered
    min_lat, max_lat = df['latitude'].min(), df['latitude'].max()
    min_lon, max_lon = df['longitude'].min(), df['longitude'].max()
    area_span = geodesic((min_lat, min_lon), (max_lat, max_lon)).miles
    
    ## Lat/lon variation
    lat_var = df['latitude'].var()
    lon_var = df['longitude'].var()

    ret = {
        'stid': df['stid'].iloc[0],
        'duration_seconds': duration,
        'total_distance_miles': total_distance,
        'mean_speed_mph': mean_speed,
        'area_span_miles': area_span,
        'lat_variation': lat_var,
        'lon_variation': lon_var,
        'behavior': df['behavior'].iloc[0]
    }
    return pd.DataFrame([ret])


def get_early_flight_segment(df: DataFrame, seconds=20):
    # Convert 'time' to datetime objects
    df['time_datetime'] = pd.to_datetime(df['time'], utc=True, format='ISO8601', errors='coerce')
    df.dropna(subset=['time_datetime'], inplace=True) # Remove rows with parsing errors

    # Convert datetime objects to numeric (Unix timestamps in seconds)
    df['time_numeric'] = df['time_datetime'].astype('int64') // 10**9

    # Find the minimum 'time_numeric' for each 'stid' using groupby and transform
    df['min_time_numeric'] = df.groupby('stid')['time_numeric'].transform('min')

    # Create a boolean mask for rows within the time window
    time_mask = df['time_numeric'] <= df['min_time_numeric'] + seconds

    # Filter the DataFrame using the mask
    result_df = df[time_mask].drop(columns=['time_datetime', 'time_numeric', 'min_time_numeric']).reset_index(drop=True)

    return result_df
    
# Get the unique behavior for each stid
def count_behaviors_df(df: DataFrame) -> int:    
    # Get unique stid-behavior pairs (since behavior is constant for each stid)
    unique_behaviors = df[["stid", "behavior"]].drop_duplicates()
    
    # Count occurrences of each behavior
    behavior_counts = unique_behaviors["behavior"].value_counts().reset_index()
    behavior_counts.columns = ["behavior", "count"]
    
    return behavior_counts

def count_behaviors_series(series: pd.Series) -> dict:
    behavior_counts = defaultdict(int)
    for behavior in series:
        behavior_counts[behavior] += 1
    return behavior_counts

## Get unique number of stids
def count_stids(df: DataFrame) -> tuple[int, np.ndarray]:
    # Get the number of unique stids
    unique_stid_count = df["stid"].nunique()
    
    # Get the list of unique stids
    unique_stids = df["stid"].unique()
    
    return unique_stid_count, unique_stids

In [4]:
atl_data = {
    "weekendPeak": make_stid_unique(pd.read_csv("../data/classified_data_atl_peakWn.csv"), 1),
    "weekendOffPeak": make_stid_unique(pd.read_csv("../data/classified_data_atl_oPeakWn.csv"), 2),
    "weekdayPeak": make_stid_unique(pd.read_csv("../data/classified_data_atl_peakWd.csv"), 3),
    "weekdayOffPeak": make_stid_unique(pd.read_csv("../data/classified_data_atl_oPeakWd.csv"), 4)
}

clt_data = {
    "weekendPeak": make_stid_unique(pd.read_csv("../data/classified_data_clt_peakWn.csv"), 5),
    "weekendOffPeak": make_stid_unique(pd.read_csv("../data/classified_data_clt_oPeakWn.csv"), 6),
    "weekdayPeak": make_stid_unique(pd.read_csv("../data/classified_data_clt_peakWd.csv"), 7),
    "weekdayOffPeak": make_stid_unique(pd.read_csv("../data/classified_data_clt_oPeakWd.csv"), 8)
}

## Print the size of all datasets
atl_total = 0
for key, df in atl_data.items():
    atl_total += len(df)
clt_total = 0
for key, df in clt_data.items():
    clt_total += len(df)
print(f"ATL dataset size: {atl_total}")
print(f"CLT dataset size: {clt_total}")

ATL dataset size: 478934
CLT dataset size: 270298


### Multilayer Perceptron

In [5]:
## Functions

header = [
    "stid", "seqNum", "latitude", "longitude", "time", "behavior"
]

combined_atl_data = pd.concat(list(atl_data.values()), ignore_index=True)
combined_clt_data = pd.concat(list(clt_data.values()), ignore_index=True)

## Change 66 and 11 to 6 and 1
combined_atl_data['behavior'] = combined_atl_data['behavior'].replace({66: 6, 11: 1})
combined_clt_data['behavior'] = combined_clt_data['behavior'].replace({66: 6, 11: 1})

print(f"ATL dataset size: {len(combined_atl_data)}")
print(f"CLT dataset size: {len(combined_clt_data)}")

ATL dataset size: 478934
CLT dataset size: 270298


In [6]:
# print(f"ATL Behavior counts{count_behaviors_df(combined_atl_data)}")
# print(f"CLT Behavior counts{count_behaviors_df(combined_clt_data)}")

print(f"ATL dataset size before trimming: {len(combined_atl_data)}")
print(f"CLT dataset size before trimming: {len(combined_clt_data)}")

ATL dataset size before trimming: 478934
CLT dataset size before trimming: 270298


In [7]:
## Trim to 5 minutes of data
time_trimmed_atl = get_early_flight_segment(combined_atl_data, seconds=300)
time_trimmed_clt = get_early_flight_segment(combined_clt_data, seconds=300)

In [8]:
# print(f"ATL Behavior counts after slimming time{count_behaviors_df(grouped_atl)}")
# print(f"CLT Behavior counts after slimming time{count_behaviors_df(grouped_clt)}")

print(f"ATL dataset size after trimming: {len(time_trimmed_atl)}")
print(f"CLT dataset size after trimming: {len(time_trimmed_clt)}")

## Group by flight
grouped_atl = time_trimmed_atl.groupby(['stid'])
grouped_clt = time_trimmed_clt.groupby(['stid'])

## Extract features
feature_list_atl = [extract_features(group) for _, group in grouped_atl]
feature_list_clt = [extract_features(group) for _, group in grouped_clt]


ATL dataset size after trimming: 353925
CLT dataset size after trimming: 160187


In [9]:
## Drop NA added because later, MLP does not accept nan values.
feature_df_atl = pd.concat(feature_list_atl, ignore_index=True).dropna()
feature_df_clt = pd.concat(feature_list_clt, ignore_index=True).dropna()

print(f"ATL Behavior counts{count_behaviors_df(feature_df_atl)}")
print(f"CLT Behavior counts{count_behaviors_df(feature_df_clt)}")

ATL Behavior counts   behavior  count
0       1.0   5985
1       6.0    203
2       2.0    176
3       5.0     35
4       4.0      6
CLT Behavior counts   behavior  count
0         1   2075
1         6    144
2         2     69
3         5     12
4         4      6


In [10]:
## Train random forest
atl_X = feature_df_atl.drop(['stid', 'behavior'], axis=1)
atl_y = feature_df_atl['behavior']

print(count_behaviors_series(atl_y))

atl_X_train, atl_X_test, atl_y_train, atl_y_test = train_test_split(atl_X, atl_y, test_size=0.2, random_state=42, stratify=atl_y)

clt_X = feature_df_clt.drop(['stid', 'behavior'], axis=1)
clt_y = feature_df_clt['behavior']

print(count_behaviors_series(clt_y))

clt_X_train, clt_X_test, clt_y_train, clt_y_test = train_test_split(clt_X, clt_y, test_size=0.2, random_state=42, stratify=clt_y)

## Class weight balenced to not specifically pick 1 more than 6
atl_random_forest = RandomForestClassifier(class_weight="balanced", random_state=42)
atl_random_forest.fit(atl_X_train, atl_y_train)

clt_random_forest = RandomForestClassifier(class_weight="balanced", random_state=42)
clt_random_forest.fit(clt_X_train, clt_y_train)

defaultdict(<class 'int'>, {1.0: 5985, 2.0: 176, 4.0: 6, 6.0: 203, 5.0: 35})
defaultdict(<class 'int'>, {1: 2075, 4: 6, 2: 69, 5: 12, 6: 144})


RandomForestClassifier(class_weight='balanced', random_state=42)

In [11]:
## Evaluation of Random Forest
print("ATL Combined Model Classification Report:")
atl_y_pred = atl_random_forest.predict(atl_X_test)
atl_y_pred_proba = atl_random_forest.predict_proba(atl_X_test)
print(classification_report(atl_y_test, atl_y_pred))

## AOC_roc score
roc_auc = roc_auc_score(atl_y_test, atl_y_pred_proba, multi_class='ovr')
print(f"ROC AUC Score: {roc_auc:.4f}")

print() 

print_model_eval(clt_random_forest, clt_X_test, clt_y_test)

ATL Combined Model Classification Report:
              precision    recall  f1-score   support

         1.0       0.99      0.99      0.99      1197
         2.0       0.77      0.77      0.77        35
         4.0       1.00      1.00      1.00         1
         5.0       0.45      0.71      0.56         7
         6.0       0.71      0.59      0.64        41

    accuracy                           0.97      1281
   macro avg       0.78      0.81      0.79      1281
weighted avg       0.97      0.97      0.97      1281

ROC AUC Score: 0.9930

Combined Model Classification Report:
              precision    recall  f1-score   support

           1       0.95      0.99      0.97       416
           2       0.83      0.71      0.77        14
           4       0.00      0.00      0.00         1
           5       1.00      0.50      0.67         2
           6       0.71      0.41      0.52        29

    accuracy                           0.94       462
   macro avg       0.70     

/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

### MLP

In [12]:
## Functions

def train_mlp_classifier_by_city(X_train, y_train) -> MLPClassifier:
    model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    return model

In [13]:
## Multilayer Precepticon
atl_mlp = train_mlp_classifier_by_city(atl_X_train, atl_y_train)
clt_mlp = train_mlp_classifier_by_city(clt_X_train, clt_y_train)

In [14]:
## Evaluate MLP
print_model_eval(atl_mlp, atl_X_test, atl_y_test)
print()
print_model_eval(clt_mlp, clt_X_test, clt_y_test)

Combined Model Classification Report:
              precision    recall  f1-score   support

         1.0       0.99      0.99      0.99      1197
         2.0       0.91      0.29      0.43        35
         4.0       0.25      1.00      0.40         1
         5.0       0.00      0.00      0.00         7
         6.0       0.41      0.73      0.53        41

    accuracy                           0.95      1281
   macro avg       0.51      0.60      0.47      1281
weighted avg       0.96      0.95      0.95      1281

ROC AUC Score: 0.9746

Combined Model Classification Report:
              precision    recall  f1-score   support

           1       0.95      0.99      0.97       416
           2       0.56      0.64      0.60        14
           4       0.00      0.00      0.00         1
           5       1.00      1.00      1.00         2
           6       0.83      0.34      0.49        29

    accuracy                           0.94       462
   macro avg       0.67      0.6

/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

In [16]:
## Lime!
import lime
import lime.lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=atl_X_train.values,
    feature_names=atl_X_train.columns.tolist(),
    class_names=["1", "2", '3', '4', '5', '6'],  # Adjust based on your label
    mode="classification"
)

feature_importances = defaultdict(list)

# Define sample indices (e.g., 100 random instances)
np.random.seed(42)
sample_indices = np.random.choice(len(atl_X_test), size=100, replace=False)

for idx in sample_indices:
    exp = explainer.explain_instance(
        data_row=atl_X_test.iloc[idx].values,
        predict_fn=atl_mlp.predict_proba,
        num_features=10
    )

    for feature, weight in exp.as_list():
        feature_importances[feature].append(weight)

# Now summarize mean importance
average_importance = {
    feature: np.mean(weights) for feature, weights in feature_importances.items()
}

# Sort and display top features
sorted_importance = dict(sorted(average_importance.items(), key=lambda x: abs(x[1]), reverse=True))

# Print top 10
print("Top global LIME features:")
for feat, score in list(sorted_importance.items())[:10]:
    print(f"{feat}: {score:.4f}")

/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier wa

Top global LIME features:
total_distance_miles > 0.15: 0.2460
area_span_miles > 0.12: 0.1783
mean_speed_mph > 21.54: -0.1271
duration_seconds > 33.00: 0.0915
total_distance_miles <= 0.02: -0.0858
0.02 < total_distance_miles <= 0.06: -0.0837
0.06 < total_distance_miles <= 0.15: -0.0812
mean_speed_mph <= 5.15: 0.0710
area_span_miles <= 0.01: -0.0607
0.01 < area_span_miles <= 0.04: -0.0601


/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/pcaab_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier wa

### Artificial Data

In [17]:
from sklearn.utils.class_weight import compute_class_weight

from collections import Counter
print("ATL class counts:", Counter(atl_y_train))
print("CLT class counts:", Counter(clt_y_train))

ATL class counts: Counter({1.0: 4788, 6.0: 162, 2.0: 141, 5.0: 28, 4.0: 5})
CLT class counts: Counter({1: 1659, 6: 115, 2: 55, 5: 10, 4: 5})


In [19]:
from imblearn.over_sampling import SMOTE

## Have to do k_neighbors=1 because of the limited data
def train_balenced_mlp_classifier_by_city(X_train, y_train) -> MLPClassifier:
    smote = SMOTE(k_neighbors=1, random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
    model.fit(X_train_resampled, y_train_resampled)
    return model

In [20]:
## Multilayer Precepticon
atl_mlp_balenced = train_balenced_mlp_classifier_by_city(atl_X_train, atl_y_train)
clt_mlp_balenced = train_balenced_mlp_classifier_by_city(clt_X_train, clt_y_train)

In [21]:
## Evaluate MLP
print_model_eval(atl_mlp_balenced, atl_X_test, atl_y_test)
print()
print_model_eval(clt_mlp_balenced, clt_X_test, clt_y_test)

Combined Model Classification Report:
              precision    recall  f1-score   support

         1.0       1.00      0.95      0.97      1197
         2.0       0.48      0.83      0.60        35
         4.0       0.17      1.00      0.29         1
         5.0       0.30      0.86      0.44         7
         6.0       0.34      0.46      0.39        41

    accuracy                           0.93      1281
   macro avg       0.46      0.82      0.54      1281
weighted avg       0.96      0.93      0.94      1281

ROC AUC Score: 0.9688

Combined Model Classification Report:
              precision    recall  f1-score   support

           1       0.98      0.91      0.94       416
           2       0.24      0.86      0.37        14
           4       0.00      0.00      0.00         1
           5       0.67      1.00      0.80         2
           6       0.63      0.41      0.50        29

    accuracy                           0.87       462
   macro avg       0.50      0.6

## Visualise Prediction

Visualizes a specific STID. You can change this by rewriting for a specific STID or change the seed.

In [ ]:
## I Will go from the start of a sample stid, incrementing in 10 seconds, check prediction, and display it using folium.
atl_coords = [33.636667, -84.428056]
clt_coords = [35.213890, -80.943054]
seed = 52636

# ## Sample for switching sides
atl_sample_4_stid = feature_df_atl[feature_df_atl['behavior'] == 4].sample(n=1, random_state=seed)['stid'].values[0]
## Get all values with that stid in the atl data
atl_sample_4 = combined_atl_data[combined_atl_data['stid'] == atl_sample_4_stid].reset_index(drop=True)
atl_sample_4['time'] = pd.to_datetime(atl_sample_4['time'], errors='coerce')
atl_sample_4_duration = atl_sample_4['time'].max() - atl_sample_4['time'].min()
print(atl_sample_4_duration)

curr_time = timedelta(seconds=20)

while curr_time < atl_sample_4_duration:
    clear_output(wait=True)
    
    m = folium.Map(location=atl_coords, zoom_start=11)
    
    ## Get the prediction
    segment = get_early_flight_segment(atl_sample_4, seconds=curr_time.total_seconds())
    coords = atl_sample_4[atl_sample_4['time'] < atl_sample_4['time'].min() + curr_time][['latitude', 'longitude']].values
    last_coord = coords[-1]

    ## Plot the coordinates
    for coord in coords:
            folium.CircleMarker(location=coord, radius=1, weight=10).add_to(m)

    segment = extract_features(segment)
    
    X = segment.drop(['stid', 'behavior'], axis=1)
    y = segment['behavior']
    y_pred = atl_mlp.predict(X)
    y_pred_proba = atl_mlp.predict_proba(X)
    # print(f"Predicted behavior: {y_pred[0]}")
    # print(f"Predicted behavior probabilities: {y_pred_proba[0]}")
    # print(f"Actual behavior: {y.iloc[0]}")
    
    ## Add label at the last coordinate
    folium.Marker(
        location=last_coord,
        popup=folium.Popup(f"{atl_sample_4_stid} Predicted: {y_pred[0]}, Actual: {y.iloc[0]}", max_width=300, show=True)
    ).add_to(m)
    
    display(m)
    # html_path = os.path.join(output_dir, "temp_map.html")
    # m.save(html_path)

    # # Render HTML in Firefox and screenshot
    # driver.get("file://" + html_path)
    # driver.save_screenshot(os.path.join(output_dir, f"atl_4_{atl_sample_4_stid}_{int(curr_time.total_seconds())}.png"))
    
    ## Continue if user presses 'enter'
    
    ## Wait 0.5 second
    time_sleep(1)
    
    ## clear screen
    curr_time += timedelta(seconds=20)

KeyboardInterrupt: 